## 2020F 인공지능 Assignment

* 문제와 코드를 확인하고 서술식 답안은 '답안 작성란'이라고 적힌 markdown cell에 작성하고, 코드는 '코드 작성란'이라고 적힌 code cell에 작성할 것.
* 채점자가 code cell들을 실행했을 때 에러없이 실행되도록 할 것.
* 수식을 작성하고자 하는데 표현하기 어려운 경우, 수기 이미지 등을 첨부하는 것도 가능함. (`<img src='파일이름'>` 사용) 이 때 이미지 파일도 함께 제출할 것.
* 다른 학생 답안 및 코드를 copy한 것이 발견되면 0점 처리됨.

## Class Perceptron, AdalineGD, AdalineSGD

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [3]:
class Perceptron(object):
    """퍼셉트론 분류기

    매개변수
    ------------
    eta : float
      학습률 (0.0과 1.0 사이)
    n_iter : int
      훈련 데이터셋 반복 횟수
    random_state : int
      가중치 무작위 초기화를 위한 난수 생성기 시드

    속성
    -----------
    w_ : 1d-array
      학습된 가중치
    errors_ : list
      에포크마다 누적된 분류 오류

    """
    def __init__(self, eta=0.01, n_iter=50, random_state=1):
        self.eta = eta
        self.n_iter = n_iter
        self.random_state = random_state

    def fit(self, X, y):
        """훈련 데이터 학습

        매개변수
        ----------
        X : {array-like}, shape = [n_samples, n_features]
          n_samples개의 샘플과 n_features개의 특성으로 이루어진 훈련 데이터
        y : array-like, shape = [n_samples]
          타깃값

        반환값
        -------
        self : object

        """
        rgen = np.random.RandomState(self.random_state)
        self.w_ = rgen.normal(loc=0.0, scale=0.01, size=1 + X.shape[1])
        self.errors_ = []

        for _ in range(self.n_iter):
            errors = 0
            for xi, target in zip(X, y):
                update = self.eta * (target - self.predict(xi))
                self.w_[1:] += update * xi
                self.w_[0] += update
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return self

    def net_input(self, X):
        """최종 입력 계산"""
        return np.dot(X, self.w_[1:]) + self.w_[0]

    def predict(self, X):
        """단위 계단 함수를 사용하여 클래스 레이블을 반환합니다"""
        return np.where(self.net_input(X) >= 0.0, 1, -1)

In [10]:
class AdalineGD(object):
    """적응형 선형 뉴런 분류기

    매개변수
    ------------
    eta : float
      학습률 (0.0과 1.0 사이)
    n_iter : int
      훈련 데이터셋 반복 횟수
    random_state : int
      가중치 무작위 초기화를 위한 난수 생성기 시드

    속성
    -----------
    w_ : 1d-array
      학습된 가중치
    cost_ : list
      에포크마다 누적된 비용 함수의 제곱합

    """
    def __init__(self, eta=0.01, n_iter=50, random_state=1):
        self.eta = eta
        self.n_iter = n_iter
        self.random_state = random_state

    def fit(self, X, y):
        """훈련 데이터 학습

        매개변수
        ----------
        X : {array-like}, shape = [n_samples, n_features]
          n_samples 개의 샘플과 n_features 개의 특성으로 이루어진 훈련 데이터
        y : array-like, shape = [n_samples]
          타깃값

        반환값
        -------
        self : object

        """
        rgen = np.random.RandomState(self.random_state)
        self.w_ = rgen.normal(loc=0.0, scale=0.01, size=1 + X.shape[1])
        self.cost_ = []

        for i in range(self.n_iter):
            net_input = self.net_input(X)
            output = self.activation(net_input)
            errors = (y - output)
            self.w_[1:] += self.eta * X.T.dot(errors)
            self.w_[0] += self.eta * errors.sum()
            cost = (errors**2).sum() / 2.0
            self.cost_.append(cost)
        return self

    def net_input(self, X):
        """최종 입력 계산"""
        return np.dot(X, self.w_[1:]) + self.w_[0]

    def activation(self, X):
        """선형 활성화 계산"""
        return X

    def predict(self, X):
        """단위 계단 함수를 사용하여 클래스 레이블을 반환합니다"""
        return np.where(self.activation(self.net_input(X)) >= 0.0, 1, -1)

In [14]:
class AdalineSGD(object):
    """ADAptive LInear NEuron 분류기

    Parameters
    ------------
    eta : float
      학습률 (0.0과 1.0 사이)
    n_iter : int
      훈련 데이터셋 반복 횟수
    shuffle : bool (default: True)
      True로 설정하면 같은 반복이 되지 않도록 에포크마다 훈련 데이터를 섞습니다
    random_state : int
      가중치 무작위 초기화를 위한 난수 생성기 시드

    Attributes
    -----------
    w_ : 1d-array
      학습된 가중치
    cost_ : list
      모든 훈련 샘플에 대해 에포크마다 누적된 평균 비용 함수의 제곱합

    """
    def __init__(self, eta=0.01, n_iter=10, shuffle=True, random_state=None):
        self.eta = eta
        self.n_iter = n_iter
        self.w_initialized = False
        self.shuffle = shuffle
        self.random_state = random_state
        
    def fit(self, X, y):
        """훈련 데이터 학습

        Parameters
        ----------
        X : {array-like}, shape = [n_samples, n_features]
          n_samples 개의 샘플과 n_features 개의 특성으로 이루어진 훈련 데이터
        y : array-like, shape = [n_samples]
          타깃 벡터

        반환값
        -------
        self : object

        """
        self._initialize_weights(X.shape[1])
        self.cost_ = []
        for i in range(self.n_iter):
            if self.shuffle:
                X, y = self._shuffle(X, y)
            cost = []
            for xi, target in zip(X, y):
                cost.append(self._update_weights(xi, target))
            avg_cost = sum(cost) / len(y)
            self.cost_.append(avg_cost)
        return self

    def partial_fit(self, X, y):
        """가중치를 다시 초기화하지 않고 훈련 데이터를 학습합니다"""
        if not self.w_initialized:
            self._initialize_weights(X.shape[1])
        if y.ravel().shape[0] > 1:
            for xi, target in zip(X, y):
                self._update_weights(xi, target)
        else:
            self._update_weights(X, y)
        return self

    def _shuffle(self, X, y):
        """훈련 데이터를 섞습니다"""
        r = self.rgen.permutation(len(y))
        return X[r], y[r]
    
    def _initialize_weights(self, m):
        """랜덤한 작은 수로 가중치를 초기화합니다"""
        self.rgen = np.random.RandomState(self.random_state)
        self.w_ = self.rgen.normal(loc=0.0, scale=0.01, size=1 + m)
        self.w_initialized = True
        
    def _update_weights(self, xi, target):
        """아달린 학습 규칙을 적용하여 가중치를 업데이트합니다"""
        output = self.activation(self.net_input(xi))
        error = (target - output)
        self.w_[1:] += self.eta * xi.dot(error)
        self.w_[0] += self.eta * error
        cost = 0.5 * error**2
        return cost
    
    def net_input(self, X):
        """최종 입력 계산"""
        return np.dot(X, self.w_[1:]) + self.w_[0]

    def activation(self, X):
        """선형 활성화 계산"""
        return X

    def predict(self, X):
        """단위 계단 함수를 사용하여 클래스 레이블을 반환합니다"""
        return np.where(self.activation(self.net_input(X)) >= 0.0, 1, -1)

**1. 클래스 Perceptron, AdalineGD, AdalineSGD에는 weight(`w_`)이 업데이트 되는 코드가 각각 두 줄씩 존재한다. 해당 코드에 포함된 변수들(update, xi, eta, X 등)의 역할 및 의미를 포함하여 수식을 설명하시오. (9점)**

답안 작성란

------------------------------------------------------------------------------------------------------------------------------------------

**2. 아래 iris 데이터(`df`)의 1, 2번 column을 input feature로 사용하여 4번 column의 class label을 <font color='red'>Iris-versicolor인 것과 Iris-virginica인 것</font> 두 class로 예측하고자 한다. 위에서 주어진 AdalineSGD 클래스와 iris 데이터를 사용하여 가중치를 학습해보고 (파라미터는 default 값 사용), 한 epoch가 지날 때마다 cost가 어떻게 변화하는지를 아래 이미지와 같이 weight 두 개를 x, y축으로 하는 그래프로 표현해보자. (13점)**

In [16]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None)

* Train/test 나누지 않고 전체 사용
* 주어진 AdalineSGD 클래스를 수정하는 경우 클래스 전체 코드를 아래에 포함할 것. 
* weight의 개수가 n개인 경우, nC2개 (n!/(2!(n-2)!))의 그래프를 모두 출력할 것.
* 참고 : contour plot 예제 https://matplotlib.org/3.1.1/gallery/images_contours_and_fields/contour_demo.html
* 필수사항 : contour(등고선), cost의 변화를 나타내는 직선 & 점, x축/y축 label (weight 이름), 그래프 안에 모든 epoch의 점을 포함할 것.
* 그래프의 색상, 폰트 등은 자유이며 화살표는 필수 아님.
<img src='cost_example.png'>

In [4]:
# 2. 코드 작성란

------------------------------------------------------------------------------------------------------------------------------------------

**3. 문제 2번과 동일한 classification을 수행하고자 한다 (1, 2번 column을 input feature로 사용하여 Iris-versicolor인 것과 Iris-virginica인 것으로 분류). ch03 코드를 참고하여 iris 데이터를 training (70%)/test (30%) data로 나누고 표준화된 training data로 LogisticRegression 모델을 학습한 뒤, ch06 코드를 참고하여 Test data에 대한 ROC curve를 그려보자. (13점)**
* Scikit-learn의 LogisticRegression 사용 (파라미터는 default 값 사용)
* 각 샘플에 대한 class 예측 probability는 predict_proba 메소드를 활용 (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression.predict_proba)

In [ ]:
# 3. 코드 작성란